In [1]:
import logging.handlers
from pathlib import Path
import os
import sys
import logging
from typing import Dict
import requests
import json
from datetime import date
from sqlalchemy import create_engine, select
from sqlalchemy.orm import Session
from tqdm.auto import tqdm
from data_models import WoWCharacter, GearLog, CharacterProgress, Base
import wow_api_models as wow


In [2]:

log = logging.getLogger(__name__)
logging.StreamHandler(stream=sys.stdout)
logging.basicConfig(encoding="utf-8", level=logging.DEBUG)
logging.getLogger("urllib3.connectionpool").level = logging.WARNING
logging.getLogger("sqlalchemy.engine.Engine").level = logging.WARNING


In [3]:
OAUTH_TOKEN = ""

list_of_characters = [
    "us|icecrown|littlegizmo",
    "us|icecrown|evilgizmo",
    "us|icecrown|tidalgizmo",
    "us|icecrown|stockygizmo",
    "us|icecrown|scaledgizmo",
    "us|icecrown|naturalgizmo",
    "us|icecrown|voidedgizmo",
    "us|icecrown|fuzzygizmo",
    "us|icecrown|felgizmo",
    "us|icecrown|tallgizmo",
    "us|icecrown|ralethisolur"
]

client_id = os.getenv("WOW_CLIENT_ID", "NeedAKey")
client_secret = os.getenv("WOW_CLIENT_SECRET", "OrThisWillNotWork")

In [4]:
def get_oauth_token(id: str, secret: str) -> Dict:
    base_url = "https://oauth.battle.net/token"
    data = {
        "grant_type": "client_credentials",
    }

    response = requests.post(base_url, data=data, auth=(id, secret))
    return response.json()


In [5]:
def get_equipment_for_character(
    region: str,
    realm: str,
    character: str,
    locale: str = "en_US",
):
    global OAUTH_TOKEN, log
    if region == "cn":
        API_HOST = "gateway.battlenet.com.cn"
    elif region in ["us", "eu", "kr", "tw"]:
        API_HOST = f"{region}.api.blizzard.com"
    else:
        raise Exception("Invalid region specified.", region)

    auth = {"Authorization": f"Bearer {OAUTH_TOKEN}"}
    realmSlug = realm.lower().strip()
    characterName = character.lower().strip()

    endpoint = f"/profile/wow/character/{realmSlug}/{characterName}/equipment"

    params = {":region": region, "namespace": f"profile-{region}", "locale": locale}

    resp = requests.get(
        url=f"https://{API_HOST}/{endpoint}",
        params=params,
        headers=auth
    )
    return resp.json()


In [6]:
def get_engine():

    db_host = os.getenv("DB_HOST")
    db_user = os.getenv("DB_USER")
    db_port = os.getenv("DB_PORT")
    db_pass = os.getenv("DB_PASS")
    db_name = os.getenv("DB_NAME")

    return create_engine(
        f"postgresql+psycopg://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}",
        echo=False
    )


In [7]:
def register_character(character_key: str, db_sess: Session):
    target = db_sess.scalar(
        select(WoWCharacter)
        .where(WoWCharacter.key == character_key)
    )
    if target is not None:
        return target
    
    region, realm, name = character_key.split('|')
    target = WoWCharacter(
        key=character_key,
        region=region,
        realm=realm,
        name=name,
        level=None,
        id=None
    )
    db_sess.add(target)
    db_sess.commit()

    return target

In [8]:
# Setup
OAUTH_TOKEN = get_oauth_token(client_id, client_secret)["access_token"]

output_dir = Path('.', 'storage', 'equipment')


In [9]:
engine = get_engine()
Base.metadata.create_all(engine)
db_session = Session(engine, autoflush=True)
_ = db_session.begin()

In [10]:
existing_characters = db_session.scalars(
    select(WoWCharacter)
).all()
for char in list_of_characters:
    if char not in [c.key for c in existing_characters]:
        log.debug(f'Character key: {char}')
        log.debug('... is not in list')
        log.debug(register_character(char, db_session))

existing_characters = db_session.scalars(
    select(WoWCharacter)
    .order_by(WoWCharacter.id)
).all()

In [11]:
for char in existing_characters:
    char.get_details(db_sess=db_session, token=OAUTH_TOKEN)

INFO:WoW_Retail_API_Endpoint.CharacterProfileSummary:Retrieving data for littlegizmo...
DEBUG:BaseDataModel:Persisting to DB
INFO:WoW_Retail_API_Endpoint.CharacterProfileSummary:Retrieving data for evilgizmo...
DEBUG:BaseDataModel:Persisting to DB
INFO:WoW_Retail_API_Endpoint.CharacterProfileSummary:Retrieving data for tidalgizmo...
DEBUG:BaseDataModel:Persisting to DB
INFO:WoW_Retail_API_Endpoint.CharacterProfileSummary:Retrieving data for stockygizmo...
DEBUG:BaseDataModel:Persisting to DB
INFO:WoW_Retail_API_Endpoint.CharacterProfileSummary:Retrieving data for scaledgizmo...
DEBUG:BaseDataModel:Persisting to DB
INFO:WoW_Retail_API_Endpoint.CharacterProfileSummary:Retrieving data for naturalgizmo...
DEBUG:BaseDataModel:Persisting to DB
INFO:WoW_Retail_API_Endpoint.CharacterProfileSummary:Retrieving data for voidedgizmo...
DEBUG:BaseDataModel:Persisting to DB
INFO:WoW_Retail_API_Endpoint.CharacterProfileSummary:Retrieving data for fuzzygizmo...
DEBUG:BaseDataModel:Persisting to DB
INF

AttributeError: 'CharacterProfileSummary' object has no attribute 'level'

In [ ]:
char.level

In [12]:
db_session.close()